## Pressure compensation to get the water depths during the flood experiments

In [1]:
import pandas as pd
barom = pd.read_csv('BAROM_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')

# import diver data
P1C = pd.read_csv('P1C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
P2C = pd.read_csv('P2C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
P3C = pd.read_csv('P3C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
P4C = pd.read_csv('P4C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
P5C = pd.read_csv('P5C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')
P6C = pd.read_csv('P6C_EXP23.csv', parse_dates=['Date_Time'], index_col='Date_Time')


barom_interpolated = barom.resample('10s').interpolate(method='time') # to have same sampling rate as diver 
barom_interpolated.rename(columns={'Pressure': 'Barom'}, inplace=True) # rename the pressure column name to barom

# drop temperature columns 
barom_interpolated = barom_interpolated.drop(columns=['Temperature'])
P1C = P1C.drop(columns=['Temperature'])
P2C = P2C.drop(columns=['Temperature'])
P3C = P3C.drop(columns=['Temperature'])
P4C = P4C.drop(columns=['Temperature'])
P5C = P5C.drop(columns=['Temperature'])
P6C = P6C.drop(columns=['Temperature'])

P1C

In [2]:
# merge barom onto diver data on the same timestamp
pressures_P1C = P1C.merge(barom_interpolated, left_index=True, right_index=True, how='left')
# calculate water column (over the diver) depth (in meters)
pressures_P1C['WC'] = ((pressures_P1C['Pressure'] - pressures_P1C['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures_P1C['Water_Depth'] = pressures_P1C['WC'] + 0.5 - 0.398
# re-sample to 1 minute intervals for simplicity
P1C_depth = pressures_P1C.resample('1min').mean()

P2C 

In [3]:
# merge barom onto diver data on the same timestamp
pressures_P2C = P2C.merge(barom_interpolated, left_index=True, right_index=True, how='left')
# calculate water column (over the diver) depth (in meters)
pressures_P2C['WC'] = ((pressures_P2C['Pressure'] - pressures_P2C['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures_P2C['Water_Depth'] = pressures_P2C['WC'] + 0.5 - 0.428
# re-sample to 1 minute intervals for simplicity
P2C_depth = pressures_P2C.resample('1min').mean()

P3C

In [4]:
# merge barom onto diver data on the same timestamp
pressures_P3C = P3C.merge(barom_interpolated, left_index=True, right_index=True, how='left')
# calculate water column (over the diver) depth (in meters)
pressures_P3C['WC'] = ((pressures_P3C['Pressure'] - pressures_P3C['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures_P3C['Water_Depth'] = pressures_P3C['WC'] + 0.5 - 0.293
# re-sample to 1 minute intervals for simplicity
P3C_depth = pressures_P3C.resample('1min').mean()

P4C

In [5]:
# merge barom onto diver data on the same timestamp
pressures_P4C = P4C.merge(barom_interpolated, left_index=True, right_index=True, how='left')
# calculate water column (over the diver) depth (in meters)
pressures_P4C['WC'] = ((pressures_P4C['Pressure'] - pressures_P4C['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures_P4C['Water_Depth'] = pressures_P4C['WC'] + 0.5 - 0.384
# re-sample to 1 minute intervals for simplicity
P4C_depth = pressures_P4C.resample('1min').mean()

P5C

In [6]:
# merge barom onto diver data on the same timestamp
pressures_P5C = P5C.merge(barom_interpolated, left_index=True, right_index=True, how='left')
# calculate water column (over the diver) depth (in meters)
pressures_P5C['WC'] = ((pressures_P5C['Pressure'] - pressures_P5C['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures_P5C['Water_Depth'] = pressures_P5C['WC'] + 0.5 - 0.477
# re-sample to 1 minute intervals for simplicity
P5C_depth = pressures_P5C.resample('1min').mean()

P6C

In [7]:
# merge barom onto diver data on the same timestamp
pressures_P6C = P6C.merge(barom_interpolated, left_index=True, right_index=True, how='left')
# calculate water column (over the diver) depth (in meters)
pressures_P6C['WC'] = ((pressures_P6C['Pressure'] - pressures_P6C['Barom'])*98.0665) / (1000*9.80665)
# calculate total water depth (in meters) 
pressures_P6C['Water_Depth'] = pressures_P6C['WC'] + 0.5 - 0.4
# re-sample to 1 minute intervals for simplicity
P6C_depth = pressures_P6C.resample('1min').mean()

In [9]:
# export to csv
P1C_depth.to_csv('depths/P1C_EXP23_depths.csv')
P2C_depth.to_csv('depths/P2C_EXP23_depths.csv')
P3C_depth.to_csv('depths/P3C_EXP23_depths.csv')
P4C_depth.to_csv('depths/P4C_EXP23_depths.csv')
P5C_depth.to_csv('depths/P5C_EXP23_depths.csv')
P6C_depth.to_csv('depths/P6C_EXP23_depths.csv')